# Get API.Guru Data
This Notebook is used to access the current data from the database of api.guru

In [88]:
import sys
import json
import time
import pickle
import csv

In [11]:
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess

In [3]:
import requests

## prepare azure access

In [12]:
block_blob_service = BlockBlobService(account_name='openapilp', account_key='4FHmQftrYn99IkPj4tQgGDjyjF8bOPUAu4aQhF54Frb70uI5p9p5zPQkqZRnRhVd4d4ueWAtcFd0KozqRTJtvg==') 

In [13]:
container_name ='metrics'
block_blob_service.create_container(container_name)

True

In [14]:
block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

In [15]:
def uploadFile(container_name, local_file_name, full_path_to_file):
    print(str(block_blob_service.create_blob_from_path(container_name, local_file_name, full_path_to_file)))

In [43]:
def listBlobs(contname):
    print("\nList blobs in the container")
    generator = block_blob_service.list_blobs(contname)
    for blob in generator:
        print("\t Blob name: " + blob.name)

## get metrics on api.guru data

In [4]:
metrics = requests.get('https://api.apis.guru/v2/metrics.json')

In [5]:
metrics = metrics.json()

In [6]:
metrics

{'numAPIs': 1034, 'numEndpoints': 25253, 'numSpecs': 1442}

In [7]:
metrics['dateAccess'] = time.strftime("%x")

In [8]:
metrics

{'numAPIs': 1034,
 'numEndpoints': 25253,
 'numSpecs': 1442,
 'dateAccess': '05/21/18'}

In [102]:
name = 'obj/metrics_052118.pickle'
with open(name, 'wb') as handle:
    pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

### upload data to azure for metrics

In [103]:
full_path_to_file = 'C:\\Users\\lukas\\PycharmProjects\\monimporter\\obj\\metrics_052118.pickle'

In [104]:
uploadFile(container_name, name, full_path_to_file)

In [105]:
listBlobs(container_name)


List blobs in the container
	 Blob name: obj/metrics_052118.pickle


## get metadata and specs

In [24]:
entries = requests.get('https://api.apis.guru/v2/list.json')

In [25]:
entries = entries.json()

In [38]:
with open('obj/'+ 'parsed_res_210519' + '.pkl', 'wb') as f:
        pickle.dump(entries, f, pickle.HIGHEST_PROTOCOL)

In [28]:
entries

{'1forge.com': {'added': '2017-05-30T08:34:14.000Z',
  'preferred': '0.0.1',
  'versions': {'0.0.1': {'added': '2017-05-30T08:34:14.000Z',
    'info': {'contact': {'email': 'contact@1forge.com',
      'name': '1Forge',
      'url': 'http://1forge.com'},
     'description': 'Stock and Forex Data and Realtime Quotes',
     'title': '1Forge Finance APIs',
     'version': '0.0.1',
     'x-apisguru-categories': ['financial'],
     'x-logo': {'backgroundColor': '#24292e',
      'url': 'https://api.apis.guru/v2/cache/logo/http_1forge.com_logo.png'},
     'x-origin': [{'format': 'swagger',
       'url': 'http://1forge.com/openapi.json',
       'version': '2.0'}],
     'x-preferred': True,
     'x-providerName': '1forge.com'},
    'swaggerUrl': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1/swagger.json',
    'swaggerYamlUrl': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1/swagger.yaml',
    'updated': '2017-06-27T16:49:57.000Z'}}},
 '6-dot-authentiqio.appspot.com': {'added': '2017-03-15T14

In [29]:
type(entries)

dict

### upload to azure

In [106]:
container_name2 = 'meta'
block_blob_service.create_container(container_name2)

Client-Request-ID=204741e8-5cdc-11e8-9d31-c69ded18204b Retry policy did not allow for a retry: Server-Timestamp=Mon, 21 May 2018 09:48:37 GMT, Server-Request-ID=897a3868-001e-0078-52e8-f0a659000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:897a3868-001e-0078-52e8-f0a659000000Time:2018-05-21T09:48:37.8447302Z</Message></Error>.


False

In [107]:
block_blob_service.set_container_acl(container_name2, public_access=PublicAccess.Container)

In [108]:
full_path_to_file = 'C:\\Users\\lukas\\PycharmProjects\\monimporter\\obj\\parsed_res_210519.pkl'

In [109]:
fullname = 'parsed_res_210519'

In [110]:
uploadFile(container_name2, fullname, full_path_to_file)

In [111]:
listBlobs(container_name2)


List blobs in the container
	 Blob name: parsed_res_210519


### get specs as json

In [55]:
def get_json(url):
    request = req.get(url)
    if request.status_code == 200:
        json = request.json()
    else:
        return 0
    return json

In [82]:
def urlto_dict(dicts):
    keys = list(dicts.keys())
    swagger_urls = {}
    for key in keys:
        entry = dicts[key]
        version = entry['preferred']
        url_entry = entry['versions'][version]['swaggerUrl']
        swagger_urls[key] = url_entry

    print(swagger_urls)
    return swagger_urls

In [86]:
def to_csv(dicts):
    timestr = time.strftime("%Y%m%d")
    with open('urls'+ timestr +'.csv', 'w', newline='') as f:
        fieldnames = ['API', 'Swagger URL']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        data = [dict(zip(fieldnames, [k,v])) for k, v in dicts.items()]
        writer.writerows(data)
    return True

In [85]:
def to_json(dicts):
    timestr = time.strftime("%Y%m%d")
    with open('urls'+timestr+'.json', 'w') as fp:
        json.dump(dicts,fp)

In [83]:
specsUrl = urlto_dict(entries)

{'1forge.com': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1/swagger.json', '6-dot-authentiqio.appspot.com': 'https://api.apis.guru/v2/specs/6-dot-authentiqio.appspot.com/6/swagger.json', 'adafruit.com': 'https://api.apis.guru/v2/specs/adafruit.com/2.0.0/swagger.json', 'adobe.com:aem': 'https://api.apis.guru/v2/specs/adobe.com/aem/1.3.0/swagger.json', 'adyen.com:CheckoutService': 'https://api.apis.guru/v2/specs/adyen.com/CheckoutService/32/openapi.json', 'adyen.com:PaymentService': 'https://api.apis.guru/v2/specs/adyen.com/PaymentService/30/openapi.json', 'adyen.com:RecurringService': 'https://api.apis.guru/v2/specs/adyen.com/RecurringService/25/openapi.json', 'afterbanks.com': 'https://api.apis.guru/v2/specs/afterbanks.com/3.0.0/swagger.json', 'agco-ats.com': 'https://api.apis.guru/v2/specs/agco-ats.com/v1/swagger.json', 'airport-web.appspot.com': 'https://api.apis.guru/v2/specs/airport-web.appspot.com/v1/swagger.json', 'amadeus.com': 'https://api.apis.guru/v2/specs/amadeus.com/1.2

In [84]:
specsUrl

{'1forge.com': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1/swagger.json',
 '6-dot-authentiqio.appspot.com': 'https://api.apis.guru/v2/specs/6-dot-authentiqio.appspot.com/6/swagger.json',
 'adafruit.com': 'https://api.apis.guru/v2/specs/adafruit.com/2.0.0/swagger.json',
 'adobe.com:aem': 'https://api.apis.guru/v2/specs/adobe.com/aem/1.3.0/swagger.json',
 'adyen.com:CheckoutService': 'https://api.apis.guru/v2/specs/adyen.com/CheckoutService/32/openapi.json',
 'adyen.com:PaymentService': 'https://api.apis.guru/v2/specs/adyen.com/PaymentService/30/openapi.json',
 'adyen.com:RecurringService': 'https://api.apis.guru/v2/specs/adyen.com/RecurringService/25/openapi.json',
 'afterbanks.com': 'https://api.apis.guru/v2/specs/afterbanks.com/3.0.0/swagger.json',
 'agco-ats.com': 'https://api.apis.guru/v2/specs/agco-ats.com/v1/swagger.json',
 'airport-web.appspot.com': 'https://api.apis.guru/v2/specs/airport-web.appspot.com/v1/swagger.json',
 'amadeus.com': 'https://api.apis.guru/v2/specs/amade

In [89]:
to_csv(specsUrl)
to_json(specsUrl)

In [92]:
namedict = "specsUrl21052018"
with open('obj/'+ namedict + '.pkl', 'wb') as f:
        pickle.dump(specsUrl, f, pickle.HIGHEST_PROTOCOL)

### upload to azure

In [112]:
container_name3 = 'specs'
block_blob_service.create_container(container_name3)

Client-Request-ID=3ed20cda-5cdc-11e8-8b4e-c69ded18204b Retry policy did not allow for a retry: Server-Timestamp=Mon, 21 May 2018 09:49:28 GMT, Server-Request-ID=897aa542-001e-0078-25e9-f0a659000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:897aa542-001e-0078-25e9-f0a659000000Time:2018-05-21T09:49:29.0954741Z</Message></Error>.


False

In [113]:
block_blob_service.set_container_acl(container_name3, public_access=PublicAccess.Container)

In [114]:
full_path_to_file = 'C:\\Users\\lukas\\PycharmProjects\\monimporter\\obj\\specsUrl21052018.pkl'

In [115]:
uploadFile(container_name3, namedict, full_path_to_file)

In [116]:
listBlobs(container_name3)


List blobs in the container
	 Blob name: parsed_res_210519
	 Blob name: specsUrl21052018


## look for info on the dataset

In [30]:
import pandas as pd

In [48]:
keys = list(entries.keys())
keys[-30:-1]

['whapi.com:accounts',
 'whapi.com:bets',
 'whapi.com:locations',
 'whapi.com:numbers',
 'whapi.com:sessions',
 'whapi.com:sportsdata',
 'who-hosts-this.com',
 'wikimedia.org',
 'windows.net:batch-BatchService',
 'windows.net:graphrbac',
 'winning.email',
 'winsms.co.za',
 'wmata.com:bus-realtime',
 'wmata.com:bus-route',
 'wmata.com:incidents',
 'wmata.com:rail-realtime',
 'wmata.com:rail-station',
 'wordassociations.net',
 'wordnik.com',
 'wowza.com',
 'wso2apistore.com:transform',
 'xero.com',
 'xkcd.com',
 'yunbi.com',
 'zalando.com',
 'zappiti.com',
 'zoom.us',
 'zoomconnect.com',
 'zuora.com']

In [27]:
for k in keys:
    print(type(k))
    print('aws' in k)

<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<class 'str'>
True
<

<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
True
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<class 'str'>
False
<

In [57]:
count_aws = 0
count_azure = 0
count_google = 0
count_apitore = 0
count_ms = 0
count_other = 0

stats_dict = dict()

In [58]:
for key in keys:
    if "azure" in key:
        count_azure += 1
    elif "aws" in key:
        count_aws += 1
    elif "apitore" in key:
        count_apitore += 1
    elif "google" in key:
        count_google += 1
    else:
        count_other += 1

print("azure: " + str(count_azure) + " percentage: " + str(count_azure/1033))
stats_dict['azure'] = count_azure
print("aws: " + str(count_aws) + " percentage: " + str(count_aws/1033))
stats_dict['aws'] = count_aws
print("apitore: " + str(count_apitore) + " percentage: " + str(count_apitore/1033))
stats_dict['apitore'] = count_apitore
print("google: " + str(count_google) + " percentage: " + str(count_google/1033))
stats_dict['google'] = count_google

print("else: " + str(count_other) + " percentage: " + str(count_other/1033))
stats_dict['else'] = count_other
print("total: " + str(count_azure + count_apitore + count_aws + count_google + count_other))

azure: 314 percentage: 0.30396902226524686
aws: 139 percentage: 0.1345595353339787
apitore: 42 percentage: 0.04065827686350436
google: 135 percentage: 0.13068731848983542
else: 403 percentage: 0.39012584704743464
total: 1033


In [56]:
stats_dict

{'apitore': 84, 'aws': 278, 'azure': 628, 'else': 795, 'google': 270}

In [47]:
with open('obj/'+ 'stats_dict' + '.pkl', 'wb') as f:
        pickle.dump(stats_dict, f, pickle.HIGHEST_PROTOCOL)